In [95]:
import pandas as pd
import numpy as np
import sys


In [115]:
#if (len(sys.argv) < 2) :
#    print('Test mode \n')
filename   = '..\\..\\Data_Post\\input\\sol_tec_5020000_field_structure.dat'
outputfile = '..\\..\\Data_Post\\wall_force_output\\output.dat'
#else:
#    filename   = sys.argv[1]
#    outputfile = 'output.dat'
precision = 12 # The precision in the original file
test = open(filename)
a = test.readline()
index_equal = a.find('=')
a = a[index_equal+1:]
a = a.split()
test.close()
del index_equal
print("Reading "+ filename)
A = pd.read_table(filename,skiprows=[0, 1],names=a,sep='\s+')
A.drop(['cell', 'order_cell', 'Q_CRITERIA', 'vort_x', 'vort_y', 'vort_z',
       'PRESSURE', 'n_repeat'], axis=1,inplace=True) 
A[['x','y','z']] = A[['x','y','z']].round(precision-2)
A = A.set_index(['x','y','z'])

Reading ..\..\Data_Post\input\sol_tec_5020000_field_structure.dat


In [116]:
y0 = A.index.levels[1][0]
y1 = A.index.levels[1][1]
y2 = A.index.levels[1][2]
y3 = A.index.levels[1][3]

In [117]:
U0 = A.xs(y0,axis=0,level=1)
U1 = A.xs(y1,axis=0,level=1)
U2 = A.xs(y2,axis=0,level=1)
U3 = A.xs(y3,axis=0,level=1)

dy1 = y1 - y0
dy2 = y2 - y0
dy3 = y3 - y0

B = np.array([[1.0,1.0,1.0,1.0 ],
              [0.0,dy1,dy2,dy3],
              [0.0,dy1**2/2.0,dy2**2/2.0,dy3**2/2.0],
              [0.0,dy1**3/6.0,dy2**3/6.0,dy3**3/6.0]])

B = np.mat(B)

b = np.array([0.0,1.0,0.0,0.0])

x = np.linalg.solve(B, b)

In [118]:
dU = x[0]*U0 + x[1]*U1 + x[2]*U2 + x[3]*U3 

In [119]:
mu = 3.571428571428571e-04
dU *= mu

In [124]:
dU['y'] = -1

In [126]:
dU.set_index(['y'])

,U
y,
-1,0.002455
-1,0.002094
-1,0.002004
-1,0.002393
-1,0.002994
-1,0.003343
-1,0.003668
-1,0.003703
-1,0.003744


In [120]:
(j,i) = dU.unstack().shape

In [123]:
data_str = dU.to_csv(sep=' ',header=False,index=True)
print('Writing to '+ outputfile)
output = open(outputfile,'w')
header = 'VARIABLES = x y z f'
header += '\n'
output.write(header)
header = 'ZONE F = POINT I = '+  str(i) +' J = ' + str(j) + ' \n'
output.write(header)
output.write(data_str)
output.close()
print('Job Done!')

Writing to ..\..\Data_Post\wall_force_output\output.dat
Job Done!


In [122]:
dU.unstack().shape

(189, 161)